# Custom Spectral Clustering with Iris Dataset

This notebook defines a custom spectral clustering function that supports:
- **Similarity graph**: "full", "eps", "knn", or "mknn"
- **Laplacian**: "sym" (symmetric) or "rw" (random walk)

It then loads the Iris dataset (from `datasets/iris.csv`) and applies our custom clustering approach, finishing with KMeans from scikit-learn.


In [16]:
import numpy as np
import pandas as pd
import math
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist, squareform


In [17]:
def check_symmetric(matrix, tol=1e-8):
    """
    Returns True if 'matrix' is symmetric within a given tolerance.
    """
    return np.allclose(matrix, matrix.T, atol=tol)

def spectral_clustering(dataframe, similarity_graph, laplacian, number_of_clusters, eps=None, k=None):
    """
    Custom spectral clustering function that:
      - Builds 'full', 'eps', 'knn', or 'mknn' adjacency graph
      - Uses either 'sym' (symmetric) or 'rw' (random walk) Laplacian
      - Finds eigen-decomposition (eigh if symmetric, else eig)
      - Clusters the final eigenvectors with KMeans
      - Returns (k, cluster_labels, silhouette)

    'eig' vs. 'eigh' depends on matrix symmetry, as requested.
    """
    # Pairwise distances
    dimension = dataframe.shape[0]
    dist_mat = squareform(pdist(dataframe))

    sample_size = len(dist_mat)
    
    # Set n based on proportional selection, but limit by log scaling for large datasets
    n = min(sample_size // 10, int(math.log(sample_size)))

    # Fallback values for epsilon and k
    epsilon = eps if eps else np.percentile(dist_mat, 90)
    k = k if k else int(np.sqrt(sample_size))
    
    if similarity_graph == "full":

        # Calculate local sigma
        sigmas = np.zeros(dimension)
        for i in tqdm(range(len(dist_mat)), desc="Local sigma"):
            sigmas[i] = sorted(dist_mat[i])[n]

        # Adjacency matrix with local sigmas
        adjacency_matrix = np.zeros((dimension, dimension))
        for i in tqdm(range(dimension), desc="Building Full Graph"):
            for j in range(i+1, dimension):
                d = np.exp(-1 * dist_mat[i,j]**2 / (sigmas[i] * sigmas[j]))
                adjacency_matrix[i,j] = d
                adjacency_matrix[j,i] = d

    elif similarity_graph == "eps":

        # Adjacency matrix with epsilon threshold
        adjacency_matrix = np.zeros((dimension, dimension))
        for i in tqdm(range(dimension), desc="Building Eps Graph"):
            for j in range(i+1, dimension):
                if dist_mat[i,j] < epsilon:
                    adjacency_matrix[i,j] = 1
                    adjacency_matrix[j,i] = 1

    elif similarity_graph == "knn":
        adjacency_matrix = np.zeros((dimension, dimension))
        for i in tqdm(range(dimension), desc="Building kNN Graph"):
            sorted_indices = np.argsort(dist_mat[i])
            k_nearest_indices = sorted_indices[1:k+1]  # Exclude itself
            adjacency_matrix[i, k_nearest_indices] = 1

    else:
        # Mutual kNN
        adjacency_matrix = np.zeros((dimension, dimension))
        for i in tqdm(range(dimension), desc="Building m-kNN Graph"):
            sorted_indices = np.argsort(dist_mat[i])
            k_nearest_indices = sorted_indices[1:k+1]
            for neighbor in k_nearest_indices:
                neighbor_sorted_indices = np.argsort(dist_mat[neighbor])
                if i in neighbor_sorted_indices[1:k+1]:
                    adjacency_matrix[i, neighbor] = 1
                    adjacency_matrix[neighbor, i] = 1

    # Degree matrix
    degrees = adjacency_matrix.sum(axis=1)

    # Build the chosen Laplacian
    if laplacian == "sym":
        d_inv_sqrt = np.zeros_like(degrees)
        nonzero = degrees > 0
        d_inv_sqrt[nonzero] = 1.0 / np.sqrt(degrees[nonzero])
        D_half = np.diag(d_inv_sqrt)
        laplacian_matrix_normalized = D_half @ adjacency_matrix @ D_half

    elif laplacian == "rw":
        d_inv = np.zeros_like(degrees)
        nonzero = degrees > 0
        d_inv[nonzero] = 1.0 / degrees[nonzero]
        D_inverse = np.diag(d_inv)
        laplacian_matrix_normalized = D_inverse @ adjacency_matrix

    else:
        raise ValueError("Unsupported laplacian type. Only 'sym' and 'rw' are allowed.")

    # Choose eig or eigh
    if check_symmetric(laplacian_matrix_normalized):
        e, v = np.linalg.eigh(laplacian_matrix_normalized)
    else:
        e_complex, v_complex = np.linalg.eig(laplacian_matrix_normalized)
        idx = np.argsort(np.real(e_complex))
        e = np.real(e_complex[idx])
        v = np.real(v_complex[:, idx])

    # Eigengap (first 10 for safety)
    eigengap = np.diff(e)
    optimal_number_of_clusters = np.argmax(eigengap[:10]) + 1

    # Decide number of clusters
    if number_of_clusters == "fixed2":
        current_k = 2
    elif number_of_clusters == "fixed3":
        current_k = 3
    else:
        current_k = max(optimal_number_of_clusters, 2)

    # KMeans on the last current_k eigenvectors
    X = v[:, -current_k:]
    clustering = KMeans(n_clusters=current_k, random_state=42, n_init=100)
    cluster_labels = clustering.fit_predict(X)

    sil_score_val = silhouette_score(dataframe, cluster_labels) if current_k > 1 else None
    return [(current_k, cluster_labels, sil_score_val)]


In [19]:
# Load the Iris dataset from 'datasets/iris.csv'
df_iris = pd.read_csv("datasets/iris.csv")

# Optionally drop non-feature columns if present
drop_cols = [col for col in ["target", "Index"] if col in df_iris.columns]
df_iris = df_iris.drop(columns=drop_cols, errors="ignore")

import pandas as pd

# Define all combinations of similarity_graph and Laplacian
similarities = ["full", "eps", "knn", "mknn"]
laplacians = ["sym", "rw"]

all_results = []
for sim in similarities:
    for lap in laplacians:
        # Call your spectral_clustering function
        result = spectral_clustering(
            dataframe=df_iris, 
            similarity_graph=sim, 
            laplacian=lap, 
            number_of_clusters="auto"  # "auto" uses your eigengap approach
        )
        
        # result is a list with one tuple: (k, cluster_labels, silhouette)
        k_found, labels, sil = result[0]
        
        all_results.append({
            "similarity_graph": sim,
            "laplacian": lap,
            "k_found": k_found,
            "silhouette_score": sil
        })

# Create a DataFrame to display and analyze
df_results = pd.DataFrame(all_results)
print("Per-combination results:\n", df_results, "\n")

# Calculate average silhouette across all combinations
avg_sil = df_results["silhouette_score"].mean()
print(f"Average Silhouette across all combos: {avg_sil:.4f}")



Building m-kNN Graph: 100%|██████████| 150/150 [00:00<00:00, 10680.50it/s]


Per-combination results:
   similarity_graph laplacian  k_found  silhouette_score
0             full       sym        2          0.629885
1             full        rw        2          0.629885
2              eps       sym        2          0.629885
3              eps        rw        2          0.286472
4              knn       sym        7          0.320007
5              knn        rw        7          0.320007
6             mknn       sym        8          0.314433
7             mknn        rw        8          0.329224 

Average Silhouette across all combos: 0.4325
